In [ ]:
"""
List Workspaces in GeoServer
"""

from glass.web.geosrv.ws import lst_ws

workspaces = lst_ws()

print(workspaces)

In [ ]:
"""
Create new GeoServer Workspace
"""

from glass.web.geosrv.ws import create_ws

create_ws('teste', overwrite=None)

In [ ]:
"""
Delete existing GeoServer Workspace
"""

from glass.web.geosrv.ws import del_ws

del_ws('teste')

In [ ]:
"""
List Stores in GeoServer
"""

from glass.web.geosrv.stores import lst_stores

stores = lst_stores('aulatst')

print(stores)

In [ ]:
"""
Delete GeoServer Store
"""

from glass.web.geosrv.stores import del_store

del_store('aulatst', 'osm_roads')

In [ ]:
"""
ESRI Shapefile to Store
"""

from glass.web.geosrv.stores import shp_to_store

in_shp = '/mnt/d/gwork/tstgsrv/pnt_flainar.shp'
store_name = 'pontos_viseu'
workspace  = 'tstflainar'

r = shp_to_store(in_shp, store_name, workspace)

In [ ]:
"""
Add Raster Layer to GeoServer
"""

from glass.web.geosrv.ws import create_workspace
from glass.geosrv.stores import add_rst_store
from glass.geosrv.lyrs import pub_rst_lyr

WORKSPACE = "tst_ruru"
STORE_NAME = "tst_img"
RASTER_FILE = r"D:\gis\cw_data_ruicruz\multibnd.tif"

create_workspace(WORKSPACE, conf=GEOSRV_CON)
add_rst_store(RASTER_FILE, STORE_NAME, WORKSPACE, conf=GEOSRV_CON)
pub_rst_lyr("multiband", STORE_NAME, WORKSPACE, 32629, conf=GEOSRV_CON)

In [ ]:
"""
PostGIS Tables to GeoServer Layers
"""

from glass.pys.oss import fprop
from glass.web.geosrv.ws     import create_ws
from glass.web.geosrv.stores import create_pgstore
from glass.web.geosrv.lyrs   import pub_pglyr
from glass.web.geosrv.sty    import create_style, assign_style_to_layer

ws     = 'cos'
dbname = 'gsrvcos'
tables = [
    'cim_cos_18_l1', 'cim_cos_18_l2', 'cim_cos_18_l3', 'cim_cos_18_l4_v2',
    'cmb_cos_18_l1', 'cmb_cos_18_l2', 'cmb_cos_18_l3', 'cmb_cos_18_l4_v2',
    'cos_18_l1', 'cos_18_l2', 'cos_18_l3', 'cos_18_l4_v2',
]

styles = {
    '/home/jasp/mrgis/gsrv/cos_18_l1.sld' : ['cim_cos_18_l1', 'cmb_cos_18_l1', 'cos_18_l1'],
    '/home/jasp/mrgis/gsrv/cos_18_l2.sld' : ['cim_cos_18_l2', 'cmb_cos_18_l2', 'cos_18_l2'],
    '/home/jasp/mrgis/gsrv/cos_18_l3.sld' : ['cim_cos_18_l3', 'cmb_cos_18_l3', 'cos_18_l3'],
    '/home/jasp/mrgis/gsrv/cos_18_l4.sld' : ['cim_cos_18_l4_v2', 'cmb_cos_18_l4_v2', 'cos_18_l4_v2'],
}

# Create new Workspace
create_ws(ws, overwrite=True)

# Add new store
store = ws + '_db'
create_pgstore(store, ws, dbname)

# Add new layers
for t in tables:
    pub_pglyr(ws, store, t)

# Add styles
for s in styles:
    sn = fprop(s, 'fn')
    create_style(sn, s)
    for l in styles[s]:
        assign_style_to_layer(sn, l)

In [ ]:
"""
Pre-Generalized Layers
"""

ws     = 'cos'
dbname = 'gsrvcos'
table  = 'cos_18_l4_v2'
geomc  = 'geom'
remove_cols = ['shape_leng', 'shape_area']

simplification = {'01' : 0.01, '05' : 0.05, '1' : 0.1}

In [ ]:
from glass.ng.prop.sql import cols_name
from glass.ng.sql.q import exec_write_q

In [ ]:
# Get Columns name

cols = cols_name(dbname, table)
cols.remove(geomc)
for i in remove_cols:
    cols.remove(i)
print(cols)

In [ ]:
qs = []
for s in simplification:
    qs.append((
        "CREATE TABLE {t}_{sid} AS "
        "SELECT {c}, ST_SimplifyPreserveTopology({g}, {sf}) AS {g} "
        "FROM {t}"
    ).format(
        t = table, sid=s, c=", ".join(cols),
        g=geomc, sf=str(simplification[s])
    ))
    
    qs.append((
        "CREATE INDEX {t}_{sid}_{g}_dist ON {t}_{sid} USING gist({g})"
    ).format(
        t = table, sid=s, g=geomc
    ))

In [ ]:
exec_write_q(dbname, qs, api='psql')

In [ ]:
from glass.pys.Xml   import write_xml_tree

gendic = {
    ("Generalization", "dataSourceNameSpace", "NaturalEarth",
     "dataSourceName", "PostGISLocal", "distance", "1",
    "featureName", table + "_01", "geomPropertyName", geomc) : '',
    ("Generalization", "dataSourceNameSpace", "NaturalEarth",
     "dataSourceName", "PostGISLocal", "distance", "5",
    "featureName", table + "_05", "geomPropertyName", geomc) : '',
    ("Generalization", "dataSourceNameSpace", "NaturalEarth",
     "dataSourceName", "PostGISLocal", "distance", "10",
    "featureName", table + "_1", "geomPropertyName", geomc) : ''
}

tree_order = {
    ("GeneralizationInfo", "dataSourceNameSpace", "NaturalEarth",
    "dataSourceName", "PostGISLocal", "featureName", "GeneralizedCos",
    "baseFeatureName", table, "geomPropertyName", geomc) : [
        ("Generalization", "dataSourceNameSpace", "NaturalEarth",
         "dataSourceName", "PostGISLocal", "distance", "1",
        "featureName", table + "_01", "geomPropertyName", geomc),
        ("Generalization", "dataSourceNameSpace", "NaturalEarth",
         "dataSourceName", "PostGISLocal", "distance", "5",
        "featureName", table + "_05", "geomPropertyName", geomc),
        ("Generalization", "dataSourceNameSpace", "NaturalEarth",
         "dataSourceName", "PostGISLocal", "distance", "10",
        "featureName", table + "_1", "geomPropertyName", geomc)
    ]
}


xml_tree = {
    ("GeneralizationInfos", "version", "1.0") : {
        ("GeneralizationInfo", "dataSourceNameSpace", "NaturalEarth",
        "dataSourceName", "PostGISLocal", "featureName", "GeneralizedCos",
        "baseFeatureName", table, "geomPropertyName", geomc) : gendic
    }
}

print(xml_tree)

write_xml_tree(xml_tree, '/home/jasp/mrgis/simpl_ex.xml', nodes_order=tree_order)